In [1]:
import os
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import torch.nn.functional as F
import sys
sys.path.append('../code/')
import d2lzh_pytorch as d2l

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DATA_ROOT='../data/'

In [2]:
def corr1d(X,K):
    w=K.shape[0]
    Y=torch.zeros((X.shape[0]-w+1))
    for i in range(Y.shape[0]):
        Y[i]=(X[i:i+w]*K).sum()
    return Y

In [3]:
X, K = torch.tensor([0, 1, 2, 3, 4, 5, 6]), torch.tensor([1, 2])
corr1d(X, K)


tensor([ 2.,  5.,  8., 11., 14., 17.])

In [20]:
def corr1d_multi_in(X,K):
    return torch.stack([corr1d(x,k) for x,k in zip(X,K)]).sum(dim=0)

X=torch.tensor([[0,1,2,3,4,5,6],
               [1,2,3,4,5,6,7],
               [2,3,4,5,6,7,8]])

K=torch.tensor([[1,2],[3,4],[-1,-3]])
corr1d_multi_in(X,K)

tensor([ 2.,  8., 14., 20., 26., 32.])

In [37]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d,self).__init__()
    def forward(self,x):
        return F.max_pool1d(x,kernel_size=x.shape[2])

In [38]:
batch_size = 64
train_data = d2l.read_imdb('train', data_root=os.path.join(DATA_ROOT, "aclImdb"))
test_data = d2l.read_imdb('test', data_root=os.path.join(DATA_ROOT, "aclImdb"))
vocab = d2l.get_vocab_imdb(train_data)
train_set = Data.TensorDataset(*d2l.preprocess_imdb(train_data, vocab))
test_set = Data.TensorDataset(*d2l.preprocess_imdb(test_data, vocab))
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)


100%|██████████| 12500/12500 [00:00<00:00, 37248.88it/s]


In [39]:
train_iter

In [40]:
class TextCNN(nn.Module):
    def __init__(self,vocab,embed_size,kernel_sizes,num_channels):
        super(TextCNN,self).__init__()
        self.embedding=nn.Embedding(len(vocab),embed_size)
        self.constant_embedding=nn.Embedding(len(vocab),embed_size)
        self.dropout=nn.Dropout(0.5)
        self.decoder=nn.Linear(sum(num_channels),2)
        
        self.pool=GlobalMaxPool1d()
        self.convs=nn.ModuleList()
        for c,k in zip(num_channels,kernel_sizes):
            self.convs.append(nn.Conv1d(in_channels=2*embed_size,out_channels=c,kernel_size=k))
    
    def forward(self,inputs):
        embeddings=torch.cat((
            self.embedding(inputs),
            self.constant_embedding(inputs)),dim=2)
        
        embeddings=embeddings.permute(0,2,1)
        encoding=torch.cat([self.pool(F.relu(conv(embeddings))).squeeze(-1) for conv in self.convs],dim=1)
        outputs=self.decoder(self.dropout(encoding))
        return outputs

In [41]:
embed_size, kernel_sizes, nums_channels = 100, [3, 4, 5], [100, 100, 100]
net = TextCNN(vocab, embed_size, kernel_sizes, nums_channels)

In [42]:
glove_vocab = Vocab.GloVe(name='6B', dim=100,
                        cache=os.path.join(DATA_ROOT))
net.embedding.weight.data.copy_(
    d2l.load_pretrained_embedding(vocab.itos, glove_vocab))
net.constant_embedding.weight.data.copy_(
    d2l.load_pretrained_embedding(vocab.itos, glove_vocab))
net.constant_embedding.weight.requires_grad = False


There are 21202 oov words.
There are 21202 oov words.


In [43]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)


training on  cuda
epoch 1, loss 0.4798, train acc 0.760, test acc 0.828, time 68.5 sec
epoch 2, loss 0.1608, train acc 0.861, test acc 0.870, time 68.9 sec
epoch 3, loss 0.0704, train acc 0.916, test acc 0.875, time 68.9 sec
epoch 4, loss 0.0290, train acc 0.958, test acc 0.879, time 69.1 sec
epoch 5, loss 0.0122, train acc 0.979, test acc 0.871, time 69.2 sec


In [44]:
d2l.predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great']) # positive

'positive'

In [45]:
d2l.predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad']) # negative

'negative'